In [137]:
import numpy as np

In [138]:
%%script false --no-raise-error
# Above script disables this block

# Quad X
m1 = [-1,1,1,1]
m2 = [1,-1,1,1]
m3 = [1,1,-1,1]
m4 = [-1,-1,-1,1]
motors = [m1,m2,m3,m4]

Couldn't find program: 'false'


In [139]:
# %%script false --no-raise-error
# Above script disables this block

# Hexa X
m1 = [-1, 0, -1, 1]
m2 = [1, 0, 1, 1]
m3 = [0.5,-0.866,-1, 1]
m4 = [-0.5,0.866,1, 1]
m5 = [-0.5,-0.866,1, 1]
m6 = [0.5,0.866,-1, 1]
motors = [m1,m2,m3,m4,m5,m6]

In [140]:
EA = []
for motor in motors:
    # 0 - roll, 1 - pitch, 2 - yaw, 3 - thrust
    m = [motor[0], motor[1], motor[2], motor[3]]
    EA.append(m)
CA = np.linalg.pinv(EA)

print(f"Actuator Effectiveness Matrix: \n{EA}")
print(f"Control Allocation Matrix: \n{CA}")

Actuator Effectiveness Matrix: 
[[-1, 0, -1, 1], [1, 0, 1, 1], [0.5, -0.866, -1, 1], [-0.5, 0.866, 1, 1], [-0.5, -0.866, 1, 1], [0.5, 0.866, -1, 1]]
Control Allocation Matrix: 
[[-0.33333333  0.33333333  0.16666667 -0.16666667 -0.16666667  0.16666667]
 [ 0.          0.         -0.2886836   0.2886836  -0.2886836   0.2886836 ]
 [-0.16666667  0.16666667 -0.16666667  0.16666667  0.16666667 -0.16666667]
 [ 0.16666667  0.16666667  0.16666667  0.16666667  0.16666667  0.16666667]]


In [141]:
# Control allocation to Torques
# [Tp, Tq, Tr, T] = [CA] * [u1, u2, u3, u4, u5, u6]
Tp = Tq = Tr = 0
T = 10
Torq = [Tp, Tq, Tr, T]
u_input = [0, 0, 0, 0, 0, 0]

In [142]:
CA_inv = np.linalg.pinv(CA)
print(CA_inv)

[[-1.00000000e+00 -1.12940496e-16 -1.00000000e+00  1.00000000e+00]
 [ 1.00000000e+00  3.44445297e-16  1.00000000e+00  1.00000000e+00]
 [ 5.00000000e-01 -8.66000000e-01 -1.00000000e+00  1.00000000e+00]
 [-5.00000000e-01  8.66000000e-01  1.00000000e+00  1.00000000e+00]
 [-5.00000000e-01 -8.66000000e-01  1.00000000e+00  1.00000000e+00]
 [ 5.00000000e-01  8.66000000e-01 -1.00000000e+00  1.00000000e+00]]


In [143]:
# Compute u1,... u6 for equilibrium with some thrust T
#  u_input = CA_inv * Torq
u_input = np.matmul(CA_inv, Torq)
print(u_input)


[10. 10. 10. 10. 10. 10.]


In [144]:
# Remove 1 motor/ disable 1 motor
CA_fault = CA
CA_fault[:, 0] =  0
print(CA_fault)

[[ 0.          0.33333333  0.16666667 -0.16666667 -0.16666667  0.16666667]
 [ 0.          0.         -0.2886836   0.2886836  -0.2886836   0.2886836 ]
 [ 0.          0.16666667 -0.16666667  0.16666667  0.16666667 -0.16666667]
 [ 0.          0.16666667  0.16666667  0.16666667  0.16666667  0.16666667]]


In [145]:
# Use u1,...u6 values from equilibrium for finding Torque values instead
# [Tp, Tq, Tr, T] = [CA_fault] * [u1, u2, u3, u4, u5, u6]
# Torq_fault = CA_fault * u_input
Torq_fault = np.matmul(CA_fault, u_input)
print(Torq_fault)
u_input_fault = u_input
print(u_input_fault)
u_input_fault[0] =  0
print(u_input_fault)
Torq_fault = np.matmul(CA_fault, u_input_fault)
print(Torq_fault)

[3.33333333e+00 3.55271368e-15 1.66666667e+00 8.33333333e+00]
[10. 10. 10. 10. 10. 10.]
[ 0. 10. 10. 10. 10. 10.]
[3.33333333e+00 3.55271368e-15 1.66666667e+00 8.33333333e+00]
